In [20]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [21]:
cols = ['engine_id', 'cycle'] + \
       [f'op_{i}' for i in range(1,4)] + \
       [f'sensor_{i}' for i in range(1,22)]

train = pd.read_csv("../data/train_FD004.txt", sep="\s+", header=None)
train.columns = cols

test = pd.read_csv("../data/test_FD004.txt", sep="\s+", header=None)
test.columns = cols

rul = pd.read_csv("../data/RUL_FD004.txt", header=None)

<h3>CREATE RUL FOR TRAIN</h3>

In [22]:
max_cycle = train.groupby('engine_id')['cycle'].max().reset_index()
max_cycle.columns = ['engine_id','max_cycle']

train = train.merge(max_cycle, on='engine_id')
train['RUL'] = train['max_cycle'] - train['cycle']
train.drop(columns=['max_cycle'], inplace=True)

# Cap RUL
RUL_CAP = 130
train['RUL'] = train['RUL'].clip(upper=RUL_CAP)

<h3>REMOVE CONSTANT SENSORS</h3>

In [23]:
sensor_cols = [col for col in train.columns if "sensor" in col]

nunique = train[sensor_cols].nunique()
variance = train[sensor_cols].var()

drop_cols = list(set(
    nunique[nunique == 1].index.tolist() +
    variance[variance < 1e-3].index.tolist()
))

train.drop(columns=drop_cols, inplace=True)
test.drop(columns=drop_cols, inplace=True)

<h3>Feature Scaling</h3>

In [24]:
features = [col for col in train.columns if col not in ['engine_id','cycle','RUL']]
# Freeze feature order
features = sorted(features)

print("Total Features Used:", len(features))
print("Feature List:", features)
scaler = MinMaxScaler()

train[features] = scaler.fit_transform(train[features])
test[features] = scaler.transform(test[features])

Total Features Used: 23
Feature List: ['op_1', 'op_2', 'op_3', 'sensor_1', 'sensor_10', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14', 'sensor_15', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_2', 'sensor_20', 'sensor_21', 'sensor_3', 'sensor_4', 'sensor_5', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9']


<h3>Create Sliding Windows</h3>

In [25]:
def create_sequences(df, seq_length):
    X, y = [], []

    for engine in df['engine_id'].unique():
        engine_data = df[df['engine_id'] == engine]
        engine_data = engine_data.sort_values('cycle')

        data = engine_data[features].values
        rul = engine_data['RUL'].values

        for i in range(len(data) - seq_length):
            X.append(data[i:i+seq_length])
            y.append(rul[i+seq_length])

    return np.array(X), np.array(y)

SEQ_LENGTH = 60

X_train_seq, y_train_seq = create_sequences(train, SEQ_LENGTH)

print("Sequence shape:", X_train_seq.shape)

Sequence shape: (46309, 60, 23)


<h3>Train/Validation Split</h3>

In [26]:
engine_ids = train['engine_id'].unique()
train_ids, val_ids = train_test_split(engine_ids, test_size=0.2, random_state=42)

train_df = train[train['engine_id'].isin(train_ids)]
val_df = train[train['engine_id'].isin(val_ids)]

X_train_seq, y_train_seq = create_sequences(train_df, SEQ_LENGTH)
X_val_seq, y_val_seq = create_sequences(val_df, SEQ_LENGTH)

# Scale target
y_train_seq = y_train_seq / RUL_CAP
y_val_seq = y_val_seq / RUL_CAP

<h3>Build LSTM Model</h3>

In [ ]:
# Build LSTM Model
model = Sequential()

model.add(LSTM(128, return_sequences=True,
               input_shape=(SEQ_LENGTH, len(features))))
model.add(Dropout(0.3))

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(64))
model.add(Dropout(0.2))

model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# 🔥 ADD THIS
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

<h3>Train</h3>

In [29]:
early_stop = EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=80,
    batch_size=128,
    callbacks=[early_stop]
)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

<h3>Evaluate on Test Set</h3>

In [ ]:
X_test_seq = []
y_test = []

for i, engine in enumerate(test['engine_id'].unique()):
    
    engine_data = test[test['engine_id'] == engine]
    engine_data = engine_data.sort_values('cycle')
    
    data = engine_data[features].values
    
    # If engine has less than SEQ_LENGTH cycles
    if len(data) < SEQ_LENGTH:
        padding = np.zeros((SEQ_LENGTH - len(data), data.shape[1]))
        data = np.vstack((padding, data))
    
    last_window = data[-SEQ_LENGTH:]
    
    X_test_seq.append(last_window)
    y_test.append(rul.iloc[i, 0])

X_test_seq = np.array(X_test_seq)
y_test = np.array(y_test)

pred_test = model.predict(X_test_seq)
pred_test = pred_test.flatten() * RUL_CAP

rmse = np.sqrt(mean_squared_error(y_test, pred_test))
print("FINAL TEST RMSE:", rmse)

8/8 [==============================] - 1s 25ms/step
FINAL TEST RMSE: 39.30955001849105


<h3>APR MODEL</h3>

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

# Healthy data (normal behavior)
healthy_data = train[train['RUL'] > 100][features].values

input_dim = healthy_data.shape[1]

input_layer = Input(shape=(input_dim,))
encoded = Dense(64, activation="relu")(input_layer)
encoded = Dense(32, activation="relu")(encoded)
decoded = Dense(64, activation="relu")(encoded)
output_layer = Dense(input_dim, activation="sigmoid")(decoded)

apr_model = Model(input_layer, output_layer)
apr_model.compile(optimizer=Adam(0.001), loss='mse')

apr_model.fit(
    healthy_data,
    healthy_data,
    epochs=30,
    batch_size=256,
    validation_split=0.1,
    verbose=1
)

Epoch 1/30
127/127 [==============================] - 1s 2ms/step - loss: 0.0389 - val_loss: 0.0026
Epoch 2/30
127/127 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 8.8475e-04
Epoch 3/30
127/127 [==============================] - 0s 2ms/step - loss: 5.8260e-04 - val_loss: 3.8901e-04
Epoch 4/30
127/127 [==============================] - 0s 2ms/step - loss: 3.3956e-04 - val_loss: 2.6830e-04
Epoch 5/30
127/127 [==============================] - 0s 2ms/step - loss: 2.2560e-04 - val_loss: 1.6654e-04
Epoch 6/30
127/127 [==============================] - 0s 2ms/step - loss: 1.3460e-04 - val_loss: 1.0177e-04
Epoch 7/30
127/127 [==============================] - 0s 2ms/step - loss: 9.0201e-05 - val_loss: 7.4664e-05
Epoch 8/30
127/127 [==============================] - 0s 2ms/step - loss: 7.5060e-05 - val_loss: 6.9847e-05
Epoch 9/30
127/127 [==============================] - 0s 2ms/step - loss: 6.8635e-05 - val_loss: 6.3357e-05
Epoch 10/30
127/127 [===================

<h3>Reconstruction Error & Threshold</h3>

In [ ]:
reconstructions = apr_model.predict(healthy_data)

mse = np.mean(np.square(healthy_data - reconstructions), axis=1)

threshold = np.mean(mse) + 3*np.std(mse)

print("APR Threshold:", threshold)

1129/1129 [==============================] - 1s 921us/step
APR Threshold: 0.00013064712855698145


<h3>Detect Alerts (On Test Last Cycle)</h3>

In [ ]:
test_last = test.groupby("engine_id").last().reset_index()
test_features = test_last[features].values

recon_test = apr_model.predict(test_features)

mse_test = np.mean(np.square(test_features - recon_test), axis=1)

test_last['anomaly_score'] = mse_test
test_last['alert'] = test_last['anomaly_score'] > threshold

test_last[['engine_id','anomaly_score','alert']].head()

8/8 [==============================] - 0s 1ms/step


,engine_id,anomaly_score,alert
0,1,0.000146,True
1,2,0.000097,False
2,3,0.000054,False
3,4,0.000027,False
4,5,0.000121,False


<h3>Health Index</h3>

In [ ]:


# RUL-based health
test_last['predicted_RUL'] = pred_test
test_last['rul_health'] = test_last['predicted_RUL'] / RUL_CAP

# APR-based health
test_last['apr_health'] = 1 - (test_last['anomaly_score'] / threshold)
test_last['apr_health'] = test_last['apr_health'].clip(lower=0)

# Combined Health Index
test_last['health_index'] = 0.7 * test_last['rul_health'] + \
                            0.3 * test_last['apr_health']

test_last[['engine_id','health_index']].head()

,engine_id,health_index
0,1,0.130036
1,2,0.463657
2,3,0.781921
3,4,0.690764
4,5,0.644327


In [ ]:
def severity(score, threshold):
    if score > 2 * threshold:
        return "CRITICAL"
    elif score > threshold:
        return "WARNING"
    else:
        return "NORMAL"

test_last['severity'] = test_last['anomaly_score'].apply(
    lambda x: severity(x, threshold)
)

test_last[['engine_id','severity']].head()

,engine_id,severity
0,1,WARNING
1,2,NORMAL
2,3,NORMAL
3,4,NORMAL
4,5,NORMAL


In [ ]:
import os
import joblib
import json

SAVE_PATH = "saved_models"

os.makedirs(SAVE_PATH, exist_ok=True)

# Save LSTM
model.save(os.path.join(SAVE_PATH, "lstm_rul_model.keras"))

# Save APR
apr_model.save(os.path.join(SAVE_PATH, "apr_autoencoder.keras"))

# Save scaler
joblib.dump(scaler, os.path.join(SAVE_PATH, "scaler.pkl"))

# Save threshold
with open(os.path.join(SAVE_PATH, "apr_threshold.json"), "w") as f:
    json.dump({"threshold": float(threshold)}, f)

# Save feature list
with open(os.path.join(SAVE_PATH, "feature_list.json"), "w") as f:
    json.dump(features, f)

print("✅ All models saved successfully.")

✅ All models saved successfully.


In [ ]:
print("Final Feature Count:", len(features))
print(features)

Final Feature Count: 23
['op_1', 'op_2', 'op_3', 'sensor_1', 'sensor_10', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14', 'sensor_15', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_2', 'sensor_20', 'sensor_21', 'sensor_3', 'sensor_4', 'sensor_5', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9']
